# Deployment Types

In [ ]:
# syft absolute
import syft as sy

## Dev Python Domain


Syft supports creating a Python domain in editable mode.
This is used mainly for experimental and development purposes.
In __Dev Python Domain__ the domain instance runs locally using the SQLite as the main storage.
This enables faster development and requires less recources to operate.

The __Dev Python Domain__ supports two options:
1. Memory node - full `syft` functionality __locally__, SQLite as a local storage.
2. Webserver node -  full `syft` functionality with API 

__When you need this?__ <br>
_When you want to develop Syft or try-out new funcitonality from separate branch._

__Prerequistes:__ <br>
1. Syft repository pulled from Github - [github.com/OpenMined/PySyft](https://github.com/OpenMined/PySyft)

For broader explanation refer to the notebook [01-setting-dev-mode.ipynb](https://github.com/OpenMined/PySyft/blob/dev/notebooks/tutorials/data-engineer/01-setting-up-dev-mode.ipynb)

To launch the local __Dev Python Domain__ use the following steps:

#### 1.1 Launch Dev Memory Node

In [ ]:
memory_node = sy.orchestra.launch(
    name="Arbitrary Dev Node",
    dev_mode=True,
    reset=True,
)

In [ ]:
assert memory_node is not None

#### 1.2 Launch Dev Webserver Node

In [ ]:
webserver_node = sy.orchestra.launch(
    name="Arbitrary Webserver Dev Node", dev_mode=True, reset=True, port=8081
)

In [ ]:
assert webserver_node is not None

#### 2. Login Into Nodes

In [ ]:
memory_node_client = memory_node.login(
    email="info@openmined.org", password="changethis"
)
memory_node_client

#### 3. Landing Memory and Webserver Node

In [ ]:
memory_node.land()

In [ ]:
webserver_node.land()

----

## Single Container / Enclave (TBD)

Single Container deployment is used when fast and painless deployment of `syft` with all essential functionality is needed. This deployment type contains the `syft` and SQLite as a light-weight database in a single container.

__When you need this?__ <br>
_When you quickly want to test syft in a single container._

__Prerequistes:__ <br>
1. Syft repository pulled from Github - [github.com/OpenMined/PySyft](https://github.com/OpenMined/PySyft)
1. Docker Installed - [docs.docker.com/get-docker](https://docs.docker.com/get-docker/)


#### Deploy Syft in Single Container Mode

Enter the PySyft Repository and run the following command

`docker run -it -e DEFAULT_ROOT_PASSWORD=secret -e PORT=8080 -p 8080:8080 openmined/grid-enclave:0.8.1`

----

## Full Container Stack

Syft can operate as a container stack. This setting consider deployment of following containers:
   - Backend - contains `Syft` and corresponding logic to execute code in _sync_ manner
   - Backend Stream - contains `Syft` and logic to queue message in RabbitMQ
   - Celery Worker - contains `Syft` and logic to execute message received from RabbitMQ
   - RabbitMQ - receives messages from Backend Stream and passes them into Celery Worker
   - Redis - each `syft` object has a `UUID`, and stored in Redis as a `key`/`value` pair
   - Mongo - Stores non-private metadata that are related to `grid` operation, such as __RBAC__ or `BLOB`s metadata 
   - SeaweedFS - Stores the `BLOB`s, compatible with Amazon S3 protocols
   - Jaeger - distributed end-to-end tracing

__When you need this?__ <br>
_When you need a Syft domain/gateway node locally._

__Prerequistes:__ <br>
1. Syft installed - [pypi.org/project/syft](https://pypi.org/project/syft/)
1. Hagrid installed - [pypi.org/project/syft](https://pypi.org/project/syft/)
1. Docker Installed - [docs.docker.com/get-docker](https://docs.docker.com/get-docker/)


Easiest way to launch the Full Container Stack is the `hagrid` cli tool.

Basic syntax of Hagrdi deployment command is the following: <br>
> `hagrid launch <domaim name> <node type> to <host>:<port>`

To deploy the full container stack use the following command: <br>

> `hagrid launch test_domain domain to docker:8081`

For detailed explanation of Full Container Stack deployment refer to the notebook [05-deploy-stack.ipynb](https://github.com/OpenMined/PySyft/blob/dev/notebooks/tutorials/data-engineer/05-deploy-stack.ipynb)

----

## VM Container Host

Ability to easily deploy `syft` stack to __anywhere__. By anywhere we mean an existing linux server accessible via `ssh` connection. `hagrid` cli tool can do all the hard work for us, by defining the desired system state using `ansible` and deploying all containers (defined in the previous section).

__When you need this?__ <br>
_When you need to deploy Syft domain/gateway node on a remote host, whether Virtual Machine or real Linux server._

__Prerequistes:__ <br>
1. Syft installed - [pypi.org/project/syft](https://pypi.org/project/syft/)
2. Hagrid installed - [pypi.org/project/syft](https://pypi.org/project/syft/)
3. VM accessible via SSH

Deploy Syft `domain`/`network` node to the remote VM using following command:

> `hagrid launch test_domain domain to 100.0.0.1 --username=ubuntu --auth-type=key --key-path=~/.ssh/hagrid_ssh_key`

All flags marked with `--` are optional, if not provided `hagrid` will interactively ask you to provide all necessary details. More details on `hagrid` usage can be found in following notebook [03-hagrid.ipynb](https://github.com/OpenMined/PySyft/blob/dev/notebooks/tutorials/data-engineer/03-hagrid.ipynb)

If you want to deploy to Cloud providers reffer to corresponding notebook:
- Azure - [06-deploy-to-azure.ipynb](https://github.com/OpenMined/PySyft/blob/dev/notebooks/tutorials/data-engineer/06-deploy-to-azure.ipynb)
- GCP - [07-deploy-to-gcp.ipynb](https://github.com/OpenMined/PySyft/blob/dev/notebooks/tutorials/data-engineer/07-deploy-to-gcp.ipynb)
- AWS - [08-deploy-to-aws.ipynb](https://github.com/OpenMined/PySyft/blob/dev/notebooks/tutorials/data-engineer/08-deploy-to-aws.ipynb)

>__Note__: VM Container Host supports deployment _only from Linux or MacOS_ machines, since it requires `ansible` <br> that is not supported by Windows 


----

## Gateway Nodes

Gateway Nodes are used to interconnect multiple `domain` nodes.
Essentially, `gateway` nodes use the same containers and code, although with different configurations.
`gateway` nodes do not have the Frontend and Blob storage.  

__When you need this?__ <br>
_When you need to interconnect two or more domain nodes._

__Prerequistes:__ <br>
1. Syft installed - [pypi.org/project/syft](https://pypi.org/project/syft/)
1. Hagrid installed - [pypi.org/project/syft](https://pypi.org/project/syft/)
1. Docker installed or SSH connection to VM

The `hagrid` cli can be used to deploy the `gateway` nodes, as a local container stack deployment or remote VM host deployment.

To deploy `gateway` node us the following command: <br>
> `hagrid launch <gateway_name> gateway to <host>:<gateway_port>`

Example of launching the `gateway` node called `test-gateway`: <br>
> `hagrid launch test-gateway gateway to docker:9082`


----